In [1]:
import fastf1
cache_location = 'C:\\Users\\Ahmed Essam\\development\\AI\\F1\\f1cache'
fastf1.Cache.enable_cache(cache_location)
grand_prix = 'Bahrain'
current_year = 2025
previous_year = current_year - 1

NotADirectoryError: Cache directory does not exist! Please check for typos or create it first.

In [ ]:
# Load FastF1 2024 GP race session
session_2024 = fastf1.get_session(previous_year, grand_prix, "R")
session_2024.load()

In [ ]:
session_2024.results.info()
session_2024.results.head(5)

In [ ]:
quali_2025 = fastf1.get_session(current_year, grand_prix, "Q")
quali_2025.load()
quali_2024 = fastf1.get_session(previous_year, grand_prix, "Q")
quali_2024.load()

In [ ]:
quali_2025.results.info()
quali_2025.results.head(5)

In [ ]:
session_2024.laps.info()
session_2024.laps.head(5)

In [ ]:
session_2024.results.info()
session_2024.results.head()

In [ ]:
session_2024.weather_data.info()
session_2024.weather_data.describe()
# session_2024.weather_data.head()

In [ ]:
session_2024.laps['LapTime'].describe()
average_laptimes = session_2024.laps.groupby('Driver')['LapTime'].mean().reset_index()

# Sort the drivers from fastest (lowest avg lap time) to slowest
average_laptimes = average_laptimes.sort_values(by='LapTime', ascending=True)

print(average_laptimes)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
quali24 = quali_2024.results[['DriverNumber', 'TeamName', 'Q1', 'Q2', 'Q3', 'Position', 'Abbreviation']].copy()
session24 = session_2024.results[['DriverNumber', 'Time', 'Status']].copy()

# Join X and y on 'DriverNumber'
data = pd.merge(quali24, session24, on='DriverNumber', how='inner')

max_laptime = average_laptimes['LapTime'].max()
data.loc[data['Status'] == 'Lapped', 'Time'] += max_laptime
print(data['Time'])
# Drop rows where 'Time' is NaT and get the corresponding 'Abbreviation'
missing_time_abbr = data[data['Time'].isna()]['Abbreviation']
print("Abbreviations with missing Time:", missing_time_abbr.tolist())
data = data.dropna(subset=['Time'])
X_train = data[['TeamName', 'Q1', 'Q2', 'Q3', 'Position', 'Abbreviation']].copy()
# X_train = data[[ 'Q1', 'Q2', 'Q3', 'Position', 'Abbreviation']].copy()

y_train = data[['Time']].copy()
y_train.dropna(inplace=True)
y_train['Time'] = y_train['Time'].dt.total_seconds()
# Build X_predict with Abbreviation
X_predict = quali_2025.results[['TeamName', 'Q1', 'Q2', 'Q3', 'Position', 'Abbreviation']].copy()
# X_predict = quali_2025.results[['Q1', 'Q2', 'Q3', 'Position', 'Abbreviation']].copy()

# Save the Abbreviation for later display and remove it from features
driver_abbr = X_predict['Abbreviation'].copy()
X_predict = X_predict.drop(columns=['Abbreviation'])

In [ ]:
# winner_time = y_train['Time'].iloc[0]
# y_train.iloc[1:, y_train.columns.get_loc('Time')] += winner_time
y_train.loc[0, 'Time'] = 0.0

In [ ]:
print(X_train, y_train)
print("the train std of Q2 and Q3 is: ", X_train[['Q2', 'Q3']].std())
print("the predict std of Q2 and Q3 is: ", X_predict[['Q2', 'Q3']].std())

In [ ]:
X_train = pd.get_dummies(X_train, columns=['TeamName'], prefix='Team')
X_predict = pd.get_dummies(X_predict, columns=['TeamName'], prefix='Team')
X_predict.rename(columns={'Team_Racing Bulls': 'Team_RB'}, inplace=True)

In [ ]:
X_train['Q2'] = X_train['Q2'].fillna(X_train['Q1'] + pd.Timedelta(seconds=10))
X_train['Q3'] = X_train['Q3'].fillna(X_train['Q2'] + pd.Timedelta(seconds=10))
# X_train = X_train[~X_train['Abbreviation'].isin(['ALB', 'ZHO', 'RIC'])]
X_train = X_train.drop(columns=['Abbreviation'])

In [ ]:
# Align y_train with the remaining rows in X_train
y_train = y_train.loc[X_train.index]

# Reset the index for both X_train and y_train to ensure they match
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [ ]:
X_predict['Q2'] = X_predict['Q2'].fillna(X_predict['Q1'] + pd.Timedelta(seconds=10))
X_predict['Q3'] = X_predict['Q3'].fillna(X_predict['Q2'] + pd.Timedelta(seconds=10))

In [ ]:
X_train['Q1'] = X_train['Q1'].dt.total_seconds()
X_train['Q2'] = X_train['Q2'].dt.total_seconds()
X_train['Q3'] = X_train['Q3'].dt.total_seconds()
X_predict['Q1'] = X_predict['Q1'].dt.total_seconds()
X_predict['Q2'] = X_predict['Q2'].dt.total_seconds()
X_predict['Q3'] = X_predict['Q3'].dt.total_seconds()


In [ ]:
print(pd.concat([X_train, y_train], axis=1))

In [ ]:
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.ensemble import VotingRegressor
import xgboost as xgb

# # Feature engineering
# X_train['Q1_Q2_diff'] = X_train['Q1'] - X_train['Q2']
# X_train['Q2_Q3_diff'] = X_train['Q2'] - X_train['Q3']
# X_predict['Q1_Q2_diff'] = X_predict['Q1'] - X_predict['Q2']



In [ ]:
# Initialize models
# rf = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=2, random_state=42)
# gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
rf = RandomForestRegressor(n_estimators=100,random_state=10)
xgb_model = xgb.XGBRegressor(n_estimators = 100, random_state=10)
gb = GradientBoostingRegressor(n_estimators=100, random_state=10)
# xgb_model = xgb.XGBRegressor(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42, verbosity=0)

# Create ensemble
ensemble = VotingRegressor([
    ('rf', rf),
    ('gb', gb),
    ('xgb', xgb_model)
])

# Use LeaveOneOut cross-validation due to small dataset
loo = LeaveOneOut()

# Calculate cross-validation scores for each model
models = {'Random Forest': rf, 'Gradient Boosting': gb, 'XGBoost': xgb_model, 'Ensemble': ensemble}
cv_results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train['Time'], 
                            scoring='neg_mean_absolute_error',
                            cv=loo)
    mae = -scores.mean()
    cv_results[name] = mae
    print(f'{name} CV MAE: {mae:.2f} seconds')

In [ ]:
def seconds_to_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = seconds % 60
    return f"{h:02d}:{m:02d}:{s:06.3f}"

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Standardize numerical features in X_train
numerical_features = ['Q1', 'Q2', 'Q3', 'Position']
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])

# Standardize numerical features in X_predict using the same scaler
X_predict[numerical_features] = scaler.transform(X_predict[numerical_features])

# Standardize the target variable
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)

print('Data standardization completed.')

In [ ]:
# Modify the model training and prediction code to handle scaled data
models = {'Random Forest': rf, 'Gradient Boosting': gb, 'XGBoost': xgb_model ,'Ensemble': ensemble}
predictions = {}

for name, model in models.items():
    # Fit the model with scaled data
    model.fit(X_train, y_train_scaled.ravel())
    
    # Make predictions and inverse transform to original scale
    pred_scaled = model.predict(X_predict)
    pred = y_scaler.inverse_transform(pred_scaled.reshape(-1, 1)).ravel()

    pred_adjusted =pred
    
    
    predictions[name] = pred_adjusted
    
    # Create results DataFrame
    results = pd.DataFrame({
        'Abbreviation': driver_abbr,
        'Predicted_Race_Time_sec': pred_adjusted,
        'Predicted_Race_Time': [seconds_to_time(s) for s in pred_adjusted]
    })
    
    # Sort and display results
    results_sorted = results.sort_values('Predicted_Race_Time_sec')
    print(f"\n{name} Predictions:")
    print(results_sorted[['Abbreviation', 'Predicted_Race_Time']])
    print(f"MAE from cross-validation: {cv_results[name]:.2f} seconds")

In [ ]:
# Feature importance analysis for each model that supports it
for name, model in models.items():
    if hasattr(model, 'feature_importances_'):
        print(f"\n{name} Feature Importances:")
        importances = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        })
        print(importances.sort_values('importance', ascending=False).head(10))

In [ ]:
# Get the winner (driver with lowest predicted race time)
winner_abb = results_sorted['Abbreviation'].iloc[0]

# Get the full driver name from qualifying data
winner_data = quali_2025.results[quali_2025.results['Abbreviation'] == winner_abb]
winner_name = winner_data['FullName'].iloc[0]

print(f"And the winner of the 2025 Bahrain Grand Prix is {winner_name}!")

In [ ]:
from IPython.display import Markdown

# Get the winner (driver with lowest predicted race time)
winner_abb = results_sorted['Abbreviation'].iloc[0]

# Get the full driver name from qualifying data
winner_data = quali_2025.results[quali_2025.results['Abbreviation'] == winner_abb]
winner_name = winner_data['FullName'].iloc[0]

markdown_text = f"""## 🏆 Race Winner Prediction


### And the winner of the 2025 Bahrain Grand Prix is:
# 🏎️ {winner_name}"""

display(Markdown(markdown_text))